In [1]:
import pyspark
from pyspark.sql import SparkSession


In [2]:
# create the spark session
spark = SparkSession.builder\
    .master("local[*]")\
    .appName("test")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 04:47:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read in the green and yellow taxi data

In [20]:
# read the green taxi data for 2020 and 2021
df_green = spark.read.parquet('data/pq/green/*/*')


In [5]:
# rename the columns
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [21]:
# read the yellow taxi data for 2020 and 2021
df_yellow = spark.read.parquet('data/pq/yellow/*/*')


In [7]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



### Check the column for both dataset with same name

In [22]:
set(df_green.columns)& set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

The pickup and dropoff datetime is named differently for both dataset. Hence, we can rename the column accordingly

In [23]:
# rename the pickuptime and dropoff_time column
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime','pickup_datetime')\
    .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')

In [24]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime','pickup_datetime')\
    .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

In [25]:
# check again the common columns
set(df_green.columns) & set(df_yellow.columns) 

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

You may notice the order is not preserved. We want it to follow the original sequence

In [27]:
common_columns = []

for column in df_green.columns:
    if column in df_yellow.columns:
        common_columns.append(column)

In [28]:
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

### Combine two dataset
If we want to combine this two dataset, we need to distinguish this them with `service_type`, hence we need to transform the column using the built-in function call `.lit()` in Spark

In [29]:
# import functions
from pyspark.sql import functions as F

In [30]:
df_green_sel = df_green \
            .select(common_columns)\
            .withColumn("service_type",F.lit('Green'))


In [32]:
df_yellow_sel = df_yellow\
            .select(common_columns)\
            .withColumn("service_type",F.lit('Yellow'))

In [34]:
df_yellow_sel.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- service_type: string (nullable = false)



In [35]:
## Union all the data
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [37]:
df_trips_data.groupBy("service_type").count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       Green| 2304517|
|      Yellow|39649199|
+------------+--------+



In [ ]:
### Write SQL Query in Spark